In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image

# 设置目标文件夹路径
folder_path = r"D:\红茶数据2024.0423\工业相机\自然\剪裁后"

# 获取所有 .bmp 图片文件
image_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.bmp')]

# 初始化数据存储
data_list = []

for img_file in image_files:
    img_path = os.path.join(folder_path, img_file)
    
    # 读取图像（PIL 以 RGB 格式打开）
    img = Image.open(img_path).convert("RGB")
    img_array = np.array(img)  # 转换为 NumPy 数组
    
    # 提取 R, G, B 颜色通道
    r, g, b = img_array[:, :, 0], img_array[:, :, 1], img_array[:, :, 2]
    
    # 计算 HSI（使用手动转换）
    img_norm = img_array / 255.0  # 归一化到 [0, 1]
    r_norm, g_norm, b_norm = img_norm[:, :, 0], img_norm[:, :, 1], img_norm[:, :, 2]
    
    # 计算亮度 (I)
    intensity = (r_norm + g_norm + b_norm) / 3.0
    
    # 计算饱和度 (S)
    min_rgb = np.minimum(np.minimum(r_norm, g_norm), b_norm)
    saturation = 1 - (3 / (r_norm + g_norm + b_norm + 1e-6)) * min_rgb  # 避免除零
    
    # 计算色调 (H)
    num = 0.5 * ((r_norm - g_norm) + (r_norm - b_norm))
    den = np.sqrt((r_norm - g_norm) ** 2 + (r_norm - b_norm) * (g_norm - b_norm)) + 1e-6  # 避免除零
    theta = np.arccos(num / den)  # 计算角度
    hue = np.degrees(theta)  # 转换为角度制
    hue[b > g] = 360 - hue[b > g]  # 调整范围
    
    # 计算 LAB 颜色空间
    from skimage import color
    img_lab = color.rgb2lab(img_array / 255.0)  # 归一化后转换
    l, a, lab_b = img_lab[:, :, 0], img_lab[:, :, 1], img_lab[:, :, 2]
    
    # 计算均值
    mean_rgb = [np.mean(r), np.mean(g), np.mean(b)]
    mean_hsi = [np.mean(hue), np.mean(saturation), np.mean(intensity)]
    mean_lab = [np.mean(l), np.mean(a), np.mean(lab_b)]
    
    # 存入列表
    data_list.append([img_file] + mean_rgb + mean_hsi + mean_lab)

# 转换为 DataFrame
columns = ["Filename", "R", "G", "B", "H", "S", "I", "L", "A", "B"]
df = pd.DataFrame(data_list, columns=columns)

# 保存到 CSV
output_csv = os.path.join(folder_path, "color_features.csv")
df.to_csv(output_csv, index=False, encoding='utf-8-sig')

print(f"颜色特征提取完成！结果已保存至 {output_csv}")


颜色特征提取完成！结果已保存至 D:\红茶数据2024.0423\工业相机\自然\剪裁后\color_features_pil.csv
